# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from prettytable import PrettyTable

#### Creating list of filepaths to process original event csv data files

In [2]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Create the data model for the Apache Cassandra and load

## The CSV file <font color=red>event_datafile_new.csv</font> has been created and is ready to load.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data looks like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Create a Cluster, Keyspace, and Tables in Apache Cassandra

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create keyspace music_library
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_library
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# Set the keyspace
try:
    session.set_keyspace('music_library')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. These queries dictate the data model of the Cassadra tables

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

#### Note: the ordering will be achieved by a clustering column on item_in_session
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [8]:
# Create all tables, one per query
queries = [
    """CREATE TABLE IF NOT EXISTS session_item (
        session_id int,
        item_in_session int,
        artist_name text,
        song_name text,
        length decimal,
        PRIMARY KEY (session_id, item_in_session)
    )""",
    """CREATE TABLE IF NOT EXISTS user_session (
        user_id int,
        session_id int,
        item_in_session int,
        artist_name text,
        song_name text,
        first_name text,
        last_name text,
        PRIMARY KEY((user_id, session_id), item_in_session)
    )""",
    """CREATE TABLE IF NOT EXISTS song_users (
        song_name text,
        user_id int,
        first_name text,
        last_name text,
        PRIMARY KEY (song_name, user_id)
    )"""
]

for query in queries:
    # Execute the query
    try:
        session.execute(query)
    except Exception as e:
        print(e)                    

In [9]:
# Scan through the csv file, map the columns, and build insert statements for each query
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        # map the columns
        artist_name = line[0]
        first_name = line[1]
        item_in_session = int(line[3])
        last_name = line[4]
        length = float(line[5])
        session_id = int(line[8])
        song_name = line[9]
        user_id = int(line[10])
                
        # Build the insert statement for query 1
        query1 = "INSERT INTO session_item (session_id, item_in_session, artist_name, song_name, length)"
        query1 = query1 + " VALUES (%s, %s, %s, %s, %s)"
        
        # Insert for query1
        session.execute(query1, (session_id, item_in_session, artist_name, song_name, length))
        
        # Build the insert statement for query 2
        query2 = "INSERT INTO user_session (user_id, session_id, item_in_session, artist_name, song_name, first_name, last_name)"
        query2 = query2 + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Insert for query2
        session.execute(query2, (user_id, session_id, item_in_session, artist_name, song_name, first_name, last_name))
        
        # Build the insert statement for query 3
        query3 = "INSERT INTO song_users (song_name, user_id, first_name, last_name)"
        query3 = query3 + " VALUES (%s, %s, %s, %s)"
        
        # Insert for query3
        session.execute(query3, (song_name, user_id, first_name, last_name))

### Verify that the data have been inserted into each table by running the target queries

In [10]:
# Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
query = """
    SELECT  artist_name,
            song_name,
            length
        FROM session_item
        WHERE session_id = 338
            AND item_in_session = 4
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Use PrettyTable to format the output
x = PrettyTable()
x.field_names = ["artist_name", "song_name", "length"]
    
for row in rows:
    x.add_row([row.artist_name, row.song_name, row.length])

print(x)

+-------------+---------------------------------+----------+
| artist_name |            song_name            |  length  |
+-------------+---------------------------------+----------+
|  Faithless  | Music Matters (Mark Knight Dub) | 495.3073 |
+-------------+---------------------------------+----------+


In [11]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182
query = """
    SELECT  artist_name, 
            song_name, 
            first_name, 
            last_name 
        FROM user_session
        WHERE user_id = 10 
            AND session_id = 182
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Use PrettyTable to format the output
x = PrettyTable()
x.field_names = ["artist_name", "song_name", "first_name", "last_name"]

for row in rows:
    x.add_row([row.artist_name, row.song_name, row.first_name, row.last_name])

print(x)

+-------------------+------------------------------------------------------+------------+-----------+
|    artist_name    |                      song_name                       | first_name | last_name |
+-------------------+------------------------------------------------------+------------+-----------+
|  Down To The Bone |                  Keep On Keepin' On                  |   Sylvie   |    Cruz   |
|    Three Drives   |                     Greece 2000                      |   Sylvie   |    Cruz   |
| Sebastien Tellier |                      Kilometer                       |   Sylvie   |    Cruz   |
|   Lonnie Gordon   | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) |   Sylvie   |    Cruz   |
+-------------------+------------------------------------------------------+------------+-----------+


In [12]:
# Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    SELECT  first_name,
            last_name
        FROM song_users
        WHERE song_name = 'All Hands Against His Own'
"""

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

# Use PrettyTable to format the output
x = PrettyTable()
x.field_names = ["first_name", "last_name"]
    
for row in rows:
    x.add_row([row.first_name, row.last_name])

print(x)

+------------+-----------+
| first_name | last_name |
+------------+-----------+
| Jacqueline |   Lynch   |
|   Tegan    |   Levine  |
|    Sara    |  Johnson  |
+------------+-----------+


### Drop the tables before closing out the sessions

In [13]:
# Drop the table before closing out the sessions
tables_to_drop = ["session_item", "user_session", "song_users"]

for table in tables_to_drop:
    query = f"DROP TABLE IF EXISTS {table}"
    try:
        ret = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [14]:
session.shutdown()
cluster.shutdown()